# Wetterstation in Waldkirch abfragen

In Waldkirch hat das Schülerforschungszentrum eine Wetterstation eingerichtet. Sie misst
- die Temperatur
- die Luftfeuchtigkeit und
- den Wind.

Wir können sie aus dem Internet aus erreichen.

Wenn Daten im Internet verfügbar gemacht werden, dann häufig über eine Schnittstelle (auch API genannt, was für *Application Programming Interface* steht)

In diesem Kursteil lernen wir, wie wir die API der Wetterstation nach Daten fragen können.

In [ ]:
import requests

In [ ]:
TOKEN = "Dein Tutor gibt dir einen TOKEN"

## Letze Messung:

In [ ]:
r = requests.get(
    "https://wetter.sfz-region-freiburg.de/api/v1/last-measurement",
    params = {"key": TOKEN}
)

In [ ]:
r.json()

## Letzten 100 Messungen:

In [ ]:
r = requests.get(
    "https://wetter.sfz-region-freiburg.de/api/v1/measurements",
    params = {"key": TOKEN, "count": 11000, "step": 1000}
)

In [ ]:
data = r.json()

In [ ]:
temperature = [d["temperature"] for d in data["data"]]
wind = [d["wind"] for d in data["data"]]
humidity = [d["humidity"] for d in data["data"]]
date = [d["date"] for d in data["data"]]

In [ ]:
import pandas as pd

In [ ]:
df = pd.DataFrame({"temperature": temperature, "wind": wind, "humidity": humidity, "date": pd.to_datetime(date, utc=True, unit='s')})

In [ ]:
df[["date", "wind"]].plot(x="date")

In [ ]:
df[["date", "temperature"]].plot(x="date")

In [ ]:
from scipy.signal import savgol_filter
import matplotlib.pyplot as plt

In [ ]:
plt.figure()
plt.plot(date, temperature)
plt.plot(date, savgol_filter(temperature, 51, 3))
plt.show()

# Zusatzaufgabe

> Ermittle Ausreißer oder "kaputte" (invalide) in den Daten